# Day 7: Amplification Circuit

In [19]:
# Copy from day 5
def read(x, prog, mode=0):
    if mode == 0: # position mode
        i = prog[x]
        assert 0 <= i, i
        assert len(prog) > i, (i, prog)
        #print("RRR", i, prog)
        return prog[i]
    elif mode == 1: # immediate mode
        assert len(prog) > x, (x, prog)
        return prog[x]
    else: assert(False)
def write(x, val, prog, mode=0):
    if mode == 0: # position mode
        i = prog[x]
        assert i >= 0, i
        assert len(prog) > i, (i, prog)
        #print("write", i, val)
        prog[i] = val
    elif mode == 1: # immediate mode
        #print("write", x, val)
        prog[x] = val
    else: assert(False)
def op_parse(num):
    op = num % 100
    m = num // 100
    modes = [0,0,0]
    modes[0] = m % 10
    m = m // 10
    modes[1] = m % 10
    m = m // 10
    modes[2] = m % 10
    return op, modes
def exec(pos,prog,inputs,outputs):
    op, modes = op_parse(prog[pos])
    #print(". pos=", pos, "op=", op, modes, "prog[pos..]=", prog[pos:pos+4])
    if op == 1: # add
        x = read(pos+1, prog, modes[0])
        y = read(pos+2, prog, modes[1])
        write(pos+3, x+y, prog, modes[2])
        return pos+4
    elif op == 2: # multiply
        x = read(pos+1, prog, modes[0])
        y = read(pos+2, prog, modes[1])
        write(pos+3, x*y, prog, modes[2])
        return pos+4
    elif op == 3: # input
        write(pos+1, inputs.pop(0), prog, modes[2])
        return pos+2
    elif op == 4: # output
        val = read(pos+1, prog, modes[2])
        #print("output val=", val)
        outputs.append(val)
        return pos+2
    elif op == 5: # jump-if-true
        x = read(pos+1, prog, modes[0])
        if x == 0:
            return pos+3
        y = read(pos+2, prog, modes[1])
        return y
    elif op == 6: # jump-if-false
        x = read(pos+1, prog, modes[0])
        if x != 0:
            return pos+3
        y = read(pos+2, prog, modes[1])
        return y
    elif op == 7: # less-than
        x = read(pos+1, prog, modes[0])
        y = read(pos+2, prog, modes[1])
        write(pos+3, x<y, prog, modes[2])
        return pos+4
    elif op == 8: # equals
        x = read(pos+1, prog, modes[0])
        y = read(pos+2, prog, modes[1])
        write(pos+3, x==y, prog, modes[2])
        return pos+4
    else:
        print("Error: op=%d at pos %d in:" % (op, pos), prog)
        assert(False)

def run(prog, inputs=list(), outputs=list()):
    prog = prog[:]
    ip = 0
    while prog[ip] != 99:
        #print(">>>", ip, "op=", prog[ip])
        ip = exec(ip,prog,inputs,outputs)
    #print(prog)
    return prog[0]

In [84]:
ExampleA = "3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0"
ExampleB = "3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0"
ExampleC = "3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0"

def assert_eq(x,y):
    if x == y: return
    print(x,"!=",y)
    assert(x == y)
    
def amplify(prog,phases):
    prog = [int(x) for x in prog.split(',')]
    val = 0
    for p in phases:
        inputs = [p, val]
        outputs = list()
        run(prog, inputs, outputs)
        #print("Ran", p, val, "->", outputs)
        val = outputs[0]
    return val
assert_eq(43210, amplify(ExampleA, [4,3,2,1,0]))
assert_eq(54321, amplify(ExampleB, [0,1,2,3,4]))
assert_eq(65210, amplify(ExampleC, [1,0,4,3,2]))

Ran 4 0 -> [4]
Ran 3 4 -> [43]
Ran 2 43 -> [432]
Ran 1 432 -> [4321]
Ran 0 4321 -> [43210]
Ran 0 0 -> [5]
Ran 1 5 -> [54]
Ran 2 54 -> [543]
Ran 3 543 -> [5432]
Ran 4 5432 -> [54321]
Ran 1 0 -> [6]
Ran 0 6 -> [65]
Ran 4 65 -> [652]
Ran 3 652 -> [6521]
Ran 2 6521 -> [65210]


In [13]:
REAL = "3,8,1001,8,10,8,105,1,0,0,21,46,67,76,97,118,199,280,361,442,99999,3,9,1002,9,3,9,101,4,9,9,102,3,9,9,1001,9,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,101,5,9,9,1002,9,2,9,101,2,9,9,4,9,99,3,9,101,4,9,9,4,9,99,3,9,1001,9,4,9,102,2,9,9,1001,9,4,9,1002,9,5,9,4,9,99,3,9,102,3,9,9,1001,9,2,9,1002,9,3,9,1001,9,3,9,4,9,99,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,99"
print(amplify(REAL, [0,1,2,3,4]))

output val= 30
output val= 132
output val= 136
output val= 1420
output val= 12789
12789


In [22]:
from itertools import permutations
max = 0
for phases in permutations((0,1,2,3,4)):
    out = amplify(REAL, phases)
    if out > max:
        max = out
print(max)

67023


# Part 2

At this point I need to redesign my IntCodeVM again, because now the program state must be maintained for the next execution.
Even for the first part, it would have made sense to externalize it to a library
and I expect more IntCode puzzles.
Thus I created the [IntCodeVM.py](IntCodeVM.py) module and put it there for easier reuse.

In [99]:
import IntCodeVM
I = IntCodeVM.IntCodeVM

vm = I(ExampleA)

In [100]:
from importlib import reload
reload(IntCodeVM)

def amplify(prog, phases):
    prog = [int(x) for x in prog.split(',')]
    vms = [I(prog) for x in range(5)]
    vms[0].inputs = [phases[0], 0]
    for i in range(4):
        vms[i].outputs = vms[i+1].inputs = list((phases[i+1],))
    vms[4].outputs = list()
    for vm in vms:
        #print("B:", vm.inputs, vm.outputs)
        b = len(vm.outputs)
        while len(vm.outputs) == b:
            vm.step()
        #print("A:", vm.inputs, vm.outputs)
    return vms[-1].outputs[-1]
assert_eq(43210, amplify(ExampleA, [4,3,2,1,0]))
assert_eq(54321, amplify(ExampleB, [0,1,2,3,4]))
assert_eq(65210, amplify(ExampleC, [1,0,4,3,2]))

Now the real part 2 stuff

In [104]:
Example2A = "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5"
Example2B = "3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10"

def amplify_feedback(prog, phases):
    prog = [int(x) for x in prog.split(',')]
    vms = [I(prog) for x in range(5)]
    for i in range(5):
        vms[i].outputs = vms[(i+1) % 5].inputs = list((phases[(i+1) % 5],))
    vms[0].inputs.append(0) # initial value
    while True:
        for i, vm in enumerate(vms):
            #print(i, "B:", vm.inputs, vm.outputs)
            b = len(vm.outputs)
            while len(vm.outputs) == b and vm.prog[vm.ip] != 99:
                vm.step()
            #print(i, "A:", vm.inputs, vm.outputs)
        if vms[0].prog[vms[0].ip] == 99:
            break
    return vms[-1].outputs[0]

assert_eq(139629729, amplify_feedback(Example2A, [9,8,7,6,5]))
assert_eq(18216, amplify_feedback(Example2B, [9,7,8,5,6]))

In [105]:
from itertools import permutations
max = 0
for phases in permutations((9,8,7,6,5)):
    out = amplify_feedback(REAL, phases)
    if out > max:
        max = out
print(max)

7818398
